In [3]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("XYZCorp_LendingData.txt", sep = "\t", low_memory = False)

In [5]:
df["purpose"] = df["purpose"].replace({"home_improvement":"house"})
df["purpose"].value_counts()


debt_consolidation    505392
credit_card           200144
house                  53469
other                  40949
major_purchase         16587
small_business          9785
car                     8593
medical                 8193
moving                  5160
vacation                4542
wedding                 2280
renewable_energy         549
educational              326
Name: purpose, dtype: int64

In [6]:
desc_df = df.dropna(subset = ["desc"])[["desc","purpose"]]

In [7]:
desc_sentences = desc_df["desc"].tolist()
import re
desc_sentences = [re.sub(pattern = "(Borrower added on \d{2}\S\d{2}\S\d{2} >)", string = x,repl = "") for x in desc_sentences]
desc_sentences = [re.sub(pattern = "<br/>", string = x,repl = "<br>") for x in desc_sentences]
desc_sentences = [re.sub(pattern = "<br>", string = x,repl = " ") for x in desc_sentences]

desc_sentences = [x.strip() for x in desc_sentences]
desc_df["desc_altered"] = desc_sentences

In [8]:
desc_df.iloc[5000]["desc_altered"]

'Consolidating all of my Credit Cards into one monthly Payment. Will Save me on interest charges through Credit Card Companies  Consolidating all of my Credit Cards into one monthly Payment. Will Save me on interest charges through Credit Card Companies'

In [9]:
desc_sentences = desc_df["desc"].tolist()
desc_sentences = [re.sub(pattern = "(Borrower added on \d{2}\S\d{2}\S\d{2} >)", string = x,repl = "") for x in desc_sentences]
desc_sentences = [re.sub(pattern = "<br/>", string = x,repl = "<br>") for x in desc_sentences]

In [11]:
split_sentences = [x.split("<br>") for x in desc_sentences]
split_sentences = [[y.strip() for y in x] for x in split_sentences]

split_sentences = [[y for y in x if y != ""] for x in split_sentences]
split_sentences = [[y.split(".") for y in x] for x in split_sentences]
split_sentences =  [[item for sublist in x for item in sublist] for x in split_sentences]

split_sentences = [[y.strip() for y in x] for x in split_sentences]
split_sentences = [[y for y in x if y != ""] for x in split_sentences]



In [11]:
split_sentences

[['I need to upgrade my business technologies'],
 ['I plan to use this money to finance the motorcycle i am looking at',
  'I plan to have it paid off as soon as possible/when i sell my old bike',
  'I only need this money because the deal im looking at is to good to pass up',
  'I plan to use this money to finance the motorcycle i am looking at',
  'I plan to have it paid off as soon as possible/when i sell my old bike',
  'I only need this money because the deal im looking at is to good to pass up',
  'I have finished college with an associates degree in business and its takingmeplaces'],
 ['to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace',
  "I'm very sorry to let my loan expired last time"],
 ['I plan on combining three large interest bills together and freeing up some extra each month to pay toward other bills',
  "I've always been a good payor but have found myself needing to make adjustments to my budget due to a medical scare",
  

In [12]:
for i in range(len(split_sentences)):
    if len(set(split_sentences[i])) != len(split_sentences[i]):
        split_sentences[i] = list(set(split_sentences[i]))
       

In [34]:
len(split_sentences)

121812

In [14]:
rejoined_sentences = [".".join(x) for x in split_sentences]

In [69]:
split_df = desc_df[["desc","purpose"]].copy()
split_df["desc"] = split_sentences
split_df = split_df.explode("desc")
split_df = split_df.dropna()

In [56]:
split_df.dtypes

desc       object
purpose    object
dtype: object

In [21]:
desc_df.to_pickle("desc_df.pkl")

In [96]:
import tensorflow as tf
import spacy

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from sklearn.metrics import roc_auc_score

In [102]:
remove_non_alpha = (re.sub("[^A-Za-z']+", ' ', str(row)) for row in df['desc'])

In [105]:
nlp = spacy.load("en_core_web_sm", disable = ["ner","parser"])
def cleaning(sentences):
    txt =[token.lemma_ for token in sentences if not token.is_stop]
    return ' '.join(txt)


In [106]:
txt = [cleaning(sentences) for sentences in nlp.pipe(remove_non_alpha,batch_size = 5000)]

MemoryError: Unable to allocate 337. MiB for an array with shape (306601, 288) and data type float32

In [70]:
sentences = split_df["desc"].tolist()
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
un

In [68]:
split_df[split_df.isna()]

,desc,purpose
0,NaN,NaN
1,NaN,NaN
1,NaN,NaN
1,NaN,NaN
1,NaN,NaN
...,...,...
855550,NaN,NaN
855550,NaN,NaN
855550,NaN,NaN
855550,NaN,NaN


In [94]:
sentence_lens = [[len(x.split(" ")) for x in y] for y in split_sentences]
sentence_lens = [max(x) for x in sentence_lens if len(x) > 0]
sentence_lens.sort(reverse = True)
print(sentence_lens)

[295, 200, 194, 175, 173, 173, 144, 141, 141, 136, 135, 130, 126, 122, 116, 116, 114, 112, 111, 109, 106, 104, 104, 103, 102, 102, 102, 101, 99, 99, 98, 96, 96, 95, 95, 95, 94, 94, 93, 93, 92, 92, 91, 90, 90, 88, 88, 86, 86, 84, 84, 84, 83, 82, 82, 82, 82, 82, 82, 81, 81, 81, 81, 81, 81, 80, 80, 79, 79, 78, 78, 78, 78, 78, 78, 78, 78, 77, 77, 77, 77, 77, 76, 76, 76, 76, 76, 75, 75, 75, 75, 75, 75, 74, 74, 74, 73, 73, 73, 73, 73, 73, 73, 72, 72, 72, 72, 72, 72, 72, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 70, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 63, 63, 63, 63, 63, 63, 63, 63, 63,

In [95]:
MAX_SEQUENCE_LENGTH = 99
data = pad_sequences(sequences, maxlen = MAX_SEQUENCE_LENGTH)
print(data)

[[    0     0     0 ...     4   115  4628]
 [    0     0     0 ...     3   432 18344]
 [    0     0     0 ...     4   286  1246]
 ...
 [    0     0     0 ...   845    17   676]
 [    0     0     0 ...    79   172   133]
 [    0     0     0 ...  1187    12   738]]
